In [ ]:
import numpy as np
import torch
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image, ImageDraw
import cv2
from torchvision.utils import save_image
from true_classify import *
from Utils import *
from anonymization_methods import *
from datasets import *
from torchvision.transforms.functional import to_pil_image
from collections import Counter
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import xlrd
import openpyxl

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# define training and testing data directories
source_path = 'F:/Original/All FR/train'

class_names = [folder for folder in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, folder))]
num_classes = len(class_names)

file_list = os.listdir(source_path)
model_dir = 'Convnext_pretrained_younas.pt'

output_path = 'F:/Anonymized/EDI/Pixel_Masked/Face_Recognition'

save_roc_dir = 'F:/Results_Final/ROC/ROC Plots Mouth_Masked/'
excel_file_path = 'F:/Results_Final/Excel_Sheets/EDI_All_FR_Results.xlsx'

In [ ]:
# Define a new transform with additional data augmentations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

In [ ]:
model = models.convnext_base(pretrained=True)
model.classifier[2]=nn.Linear(1024,num_classes)

model.load_state_dict(torch.load('Convnext_pretrained_younas.pt'))
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.001)

In [ ]:
# We will try a fixed q in each iteration
q=95
blur_kernel_size = 5

In [ ]:
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr, roc_auc, dataset_name, save_dir):
    plt.figure()
    lw = 2

    # Plot macro-average ROC curve
    plt.plot(fpr['micro'], tpr['micro'], color='deeppink', linestyle=':', lw=lw,
             label='Macro-average ROC curve (area = {0:0.2f})'.format(roc_auc['micro']))

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC for ' + dataset_name)
    plt.legend(loc="lower right")

    plt.savefig(os.path.join(save_dir, f'ROC_{dataset_name}.png'))
    plt.close()

In [ ]:
def draw_points_on_image(image, important_pixels_mask, point_size, point_color):
    image = image.to(device)
    important_pixels_mask = important_pixels_mask.to(device)

    # Apply the mask directly on the GPU tensors
    y_indices, x_indices = torch.nonzero(important_pixels_mask, as_tuple=True)
    
    image_pil = transforms.ToPILImage()(image.squeeze().cpu())
    draw = ImageDraw.Draw(image_pil)

    for y, x in zip(y_indices, x_indices):
        x, y = x.item(), y.item()
        x0, y0 = x - point_size, y - point_size
        x1, y1 = x + point_size, y + point_size
        bbox = [(x0, y0), (x1, y1)]

        # Draw a point at the important pixel
        draw.rectangle(bbox, fill=point_color)

    return transforms.ToTensor()(image_pil).unsqueeze(0)


In [ ]:
def save_image(annonymized_image, i, correct_label, output_path, q):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    ts.save(annonymized_image, os.path.join(output_path, f"{correct_label.item()}-{i}-q{q}.jpg"))

In [ ]:
start_time = time.time()
new_batch_size = 1
iteration_out_path = 'F:/Anonymized/EDI/Pixel_Masked/Face_Recognition'
new_test_path = source_path #'F:/Anonymized/EDI/Pixel_Masked/Face_Recognition/Iteration_38'
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
prev_acc = final_acc
print(prev_acc*100)
end_time = time.time()

acc_time = end_time - start_time

print(f"Acc time is", acc_time)

In [ ]:
start_time = time.time()

for itera in range(3):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)

        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"

        class_label = labels[i]
        class_subfolder = class_names[class_label]            

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True

        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)


        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data
            
            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)
            
            # Call the draw_dots_on_image function
            annonymized_image = draw_points_on_image(annonymized_image, optimized_gradients[0, 0, :, :] >= 1, point_size=0.05, point_color=(255, 0, 0))

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, q)
    
    torch.cuda.empty_cache()
    
    our_test_loader = create_test_loader(iteration_out_path, batch_size=1)
    accuracy, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
    print(f"\nAccuracy: {accuracy*100} %")

end_time = time.time()
Anon_execution_time = end_time - start_time
print(f"\nAnon Time: {Anon_execution_time} seconds")
print(f"All images processed for iteration number {itera}.")

iteration 5 Accuracy: 37.46586564718733 %
iteration 6 Accuracy: 33.28782086291644 %
iteration 7 Accuracy: 30.99399235390497 %
iteration 8 Accuracy: 28.809393773894048 %
iteration 9 Accuracy: 27.2801747678864 %
iteration 10 Accuracy: 24.658656471873293 %
iteration 11 Accuracy: 23.64827962861824 %
iteration 12 Accuracy: 23.64827962861824 %
iteration 13 Accuracy: 22.200983069361005 %
iteration 14 Accuracy: 18.70562534134353 %
iteration 15 Accuracy: 18.37793555434189 %
iteration 16 Accuracy: 16.739486619333697 %
iteration 17 Accuracy: 15.319497542326596 %
iteration 18 Accuracy: 14.964500273074824 %
iteration 19 Accuracy: 14.063353358820315 %
iteration 20 Accuracy: 12.99836155106499 %
iteration 21 Accuracy: 12.261059530311305 %
iteration 22 Accuracy: 11.851447296559257 %
iteration 23 Accuracy: 11.714909885308575 %
iteration 24 Accuracy: 10.677225559803386 %
iteration 25 Accuracy: 9.85800109229929 %
iteration 26 Accuracy: 9.666848716548335 %

Accuracy: 53.1403604587657 %

Anon Time: 1753.5750188827515 seconds
All images processed for iteration number 2.

In [ ]:
# from PIL import Image

# def resize_and_save(input_path, output_path):
#     try:
#         # Open the image file
#         with Image.open(input_path) as img:
#             # Resize the image
#             resized_img = img.resize((1024, 1024))

#             # Save the resized image
#             resized_img.save(output_path)
#             print(f"Image resized and saved to {output_path}")
#     except Exception as e:
#         print(f"Error: {e}")

# # Example usage:
# input_image_path = "D:/Younas_Work/D2_Final/Anonymized/Expite/Masked_New_2/Face_Recognition/Iteration_0/Al Pacino/Original.jpg"
# output_image_path = "D:/Younas_Work/D2_Final/Anonymized/Expite/Masked_New_2/Face_Recognition/Iteration_0/Al Pacino/Resized.jpg"

# resize_and_save(input_image_path, output_image_path )